** Scratch pad for tests **
Expects HF models to be served with TGI - use `make run_tgi` to launch TGi from docker container. 
Launch api server with `python chat_api.py`

In [ ]:
import requests
from pydantic import BaseModel
from typing import Optional
import json

In [ ]:
url = 'http://localhost:8884/create_agent_session'
response = requests.get(url)
response


In [ ]:
session=response.content.decode('utf-8')
session = json.loads(session)
session_id=session['session_id']
data = {"user_input":"What are the top 3 tourist destiantions in Sydney?","session_id":session_id}
#data
print(data)


In [ ]:
%%timeit -n 1 -r 1
url_chat = 'http://localhost:8884/chat_with_agent'
with requests.post(url_chat, json=data, stream=True) as response:
    for x in response.iter_content(chunk_size=None):
        print (x.decode('utf-8'), end="", flush=True)


In [ ]:
%%timeit -n 5 -r 1
url_chat = 'http://localhost:8884/chat_with_agent'
with requests.post(url_chat, json=data) as response:
    print (response.content)

In [ ]:
import aiohttp
import asyncio

In [ ]:
url_chat = 'http://localhost:8884/chat_with_agent'
async with aiohttp.ClientSession() as session:
    async with session.post(url_chat, json=data) as response:
        async for x in response.content.iter_any():
            print (x.decode('utf-8'), end="", flush=True)

In [ ]:
data['user_input'] = "You are good at finding and correcting spelling mistakes. Please correct the spelling mistakes to make it proper English: 'Please Sow mW capiorel for\n\nhelio.\nof gourrel Sundiicpes and hore\nmeade. deseer le'"

In [ ]:
url_chat = 'http://localhost:8884/chat_with_agent'
async with aiohttp.ClientSession() as session:
    async with session.post(url_chat, json=data) as response:
        async for x in response.content.iter_any():
            print (x.decode('utf-8'), end="", flush=True)

In [ ]:
from omegaconf import OmegaConf
from griptape.rules import Rule, Ruleset
from typing import List, Dict
from dataclasses import dataclass
from attr import field, define, Factory

@dataclass
class my_configs:
    rule_strings: List[str] = field(default=Factory(list), kw_only=True)

    @property
    def rules(self):
        rule_list=[]
        for rule in self.rule_strings:
            rule_list.append[Rule(rule)]
        return rule_list
    @rules.setter
    def rules(self,rules:List[str]):
        self.rule_strings = self.rules
        


In [ ]:
test =my_configs(rule_strings=['You are a total tool','You are not a tool'])

_config = OmegaConf.create(test)
OmegaConf.save(_config,"conf.yaml")

In [ ]:
_config.keys()

In [ ]:
a=OmegaConf.load("conf.yaml")
parsed_a = OmegaConf.structured(a,my_configs)

In [ ]:
from omegaconf import OmegaConf
from data.chat_data_classes import ModelDriverConfiguration
a=ModelDriverConfiguration(model='http://localhost:8080',pretrained_tokenizer="/home/mtman/Documents/Repos/yakwith.ai/models/Mistral-7B-OpenOrca")
conf = OmegaConf.create(a)
OmegaConf.save(conf,'default_model_driver.yaml')
conf

In [ ]:
x = OmegaConf.load('default_model_driver.yaml')

In [ ]:
x.keys()

In [ ]:
type(x), x.token

In [ ]:
for thing in x:
    print(getattr(x,thing))

In [ ]:
new_conf = ModelDriverConfiguration()

In [ ]:
type(new_conf)

In [ ]:
new_conf.from_omega_conf(x)

In [ ]:
new_conf

In [ ]:
from yak_agent import YakAgent

In [2]:
from data.chat_data_classes import RuleList
from griptape.rules import Rule
from omegaconf import OmegaConf

In [4]:
conf=OmegaConf.load('configs/rules/agent/default_rule_set.yaml')
cls=RuleList().from_omega_conf(conf)

AttributeError: 'RuleList' object attribute 'name' is read-only

In [3]:
for key in conf:
    if hasattr(cls, key):
        _value = getattr(conf, key)
        if key == "rules":
            ret = []
            for rule_string in _value:
                ret.append(Rule(rule_string))
            setattr(cls, key, ret)
        else:
            setattr(cls, key, _value)

In [8]:
cls.name

AttributeError: 'NoneType' object has no attribute 'name'

In [5]:
conf

{'name': 'CommonRules', 'rules': ['You can not substitute any ingredients', 'If a meal contains meat, you can replace meat with Tofu or Mushrooms', "If you do not know the answer to a customers questions, say 'I have to ask the manager, please give me a second.'", 'All meals and drinks can be ordered for takeaway.']}

In [30]:
a

Rule(value='Hello')